# RUN THURSDAY.ipynb
# ====================================================================
# 📊 GUT SCORE INTEGRATION PIPELINE
# Run this on Thursday to update training data with your latest ratings
# ====================================================================

In [2]:
import os
import glob
from datetime import datetime

import pandas as pd

# ====================================================================
# 🎯 THURSDAY: CLEAN OUTPUT VERSION
# Only shows NEW albums being processed
# ====================================================================

print("🗓️ RUN THURSDAY: Clean Output Version")
print("=" * 60)
print("📅 " + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print("=" * 60)

# --- STEP 1: LOAD GUT SCORES FROM MASTER FILE ---
print("\n📊 Loading gut scores from MASTER file...")

master_file = 'feedback/master_gut_scores.csv'

if not os.path.exists(master_file):
    print(f"❌ Master file not found: {master_file}")
    exit()

master_df = pd.read_csv(master_file)
master_df = master_df[master_df['gut_score'].notna()]

if len(master_df) == 0:
    print("❌ No gut scores found!")
    exit()

total_albums = len(master_df)
print(f"✅ Found {total_albums} total rated albums in master file")

# --- STEP 2: LOAD TRAINING DATA ---
print("\n📁 Loading training data...")
training_file = 'data/2026_training_complete_with_features.csv'

if not os.path.exists(training_file):
    print(f"❌ Training file not found: {training_file}")
    exit()

df_training = pd.read_csv(training_file)
print(f"  Current: {len(df_training):,} tracks")

# Count existing gut-scored tracks
if 'source_type' in df_training.columns:
    existing_gut = df_training[df_training['source_type'] == 'gut_score_rated']
    existing_count = len(existing_gut)
    print(f"  Existing gut-scored tracks: {existing_count:,}")
else:
    existing_count = 0

# --- STEP 3: FIND AND ADD ONLY NEW TRACKS ---
print("\n🎵 Processing NEW gut-scored albums...")

# Get archives (newest first)
archive_files = sorted(glob.glob('data/archived_nmf_with_features/*.csv'), reverse=True)

if not archive_files:
    print("❌ No archive files found!")
    exit()

tracks_added = 0
new_albums = 0
new_albums_list = []

for _, fb_row in master_df.iterrows():
    artist = fb_row['Artist']
    album = fb_row['Album']
    score = fb_row['gut_score']
    
    # Skip if already in training as gut_score_rated
    already_in_training = df_training[
        (df_training['Album Name'] == album) &
        (df_training['Artist Name(s)'].str.contains(artist, na=False)) &
        (df_training['source_type'] == 'gut_score_rated')
    ]
    
    if len(already_in_training) > 0:
        continue  # SILENTLY skip - don't print anything!
    
    # ---- ONLY REACH HERE FOR NEW ALBUMS ----
    if new_albums == 0:  # First new album
        print("  Processing...")
    
    new_albums += 1
    new_albums_list.append((artist, album, score))
    
    # Search archives
    found_tracks = None
    for archive_file in archive_files:
        try:
            archive_df = pd.read_csv(archive_file)
            album_tracks = archive_df[
                (archive_df['Album Name'] == album) &
                (archive_df['Artist Name(s)'].str.contains(artist, na=False))
            ].copy()
            
            if len(album_tracks) > 0:
                found_tracks = album_tracks
                break
        except:
            continue
    
    if found_tracks is None:
        print(f"  ❌ Could not find tracks for: {artist} - {album}")
        continue
    
    # Add gut score
    found_tracks['liked'] = score
    found_tracks['source_type'] = 'gut_score_rated'
    found_tracks['gut_score_date'] = datetime.now().strftime('%Y-%m-%d')
    
    # Add to training
    df_training = pd.concat([df_training, found_tracks], ignore_index=True)
    tracks_added += len(found_tracks)

# --- STEP 4: DUPLICATE HANDLING (SILENT) ---
if tracks_added > 0:
    # Silent duplicate handling
    def clean_artist_name(artist_str):
        if pd.isna(artist_str): return ""
        artist = str(artist_str).strip()
        separators = [' feat. ', ' featuring ', ' ft. ', ' with ', ' & ', ' and ', ';', ',']
        for sep in separators:
            if sep in artist.lower():
                artist = artist.split(sep)[0].strip()
        return artist
    
    df_training['artist_clean'] = df_training['Artist Name(s)'].apply(clean_artist_name)
    df_training = df_training.sort_values('liked', ascending=False)
    df_training = df_training.drop_duplicates(
        subset=['Album Name', 'artist_clean', 'Track Name'],
        keep='first'
    )
    df_training = df_training.drop(columns=['artist_clean'], errors='ignore')

# --- STEP 5: SAVE AND REPORT ---
print("\n" + "=" * 60)
print("📊 WEEKLY REPORT")
print("=" * 60)

if tracks_added > 0:
    # Create backup
    backup_dir = 'data/backups'
    os.makedirs(backup_dir, exist_ok=True)
    backup_file = f"{backup_dir}/training_backup_{datetime.now().strftime('%Y%m%d')}.csv"
    pd.read_csv(training_file).to_csv(backup_file, index=False)
    
    # Save updated training
    df_training.to_csv(training_file, index=False)
    
    print(f"✅ ADDED THIS WEEK:")
    print("-" * 60)
    for artist, album, score in new_albums_list:
        print(f"  • {artist[:25]:<25} - {album[:25]:<25} → {score}")
    
    print("-" * 60)
    print(f"  Tracks added: {tracks_added:,}")
    print(f"  Albums added: {new_albums}")
    
else:
    print("ℹ️ No new gut scores to add this week")

# Final stats
print(f"\n📈 OVERALL TOTALS:")
print("-" * 60)
print(f"  Total rated albums: {total_albums}")
print(f"  Gut-scored tracks: {len(df_training[df_training['source_type'] == 'gut_score_rated']):,}")
print(f"  Total training tracks: {len(df_training):,}")

print("\n" + "=" * 60)
print("✅ THURSDAY PROCESSING COMPLETE")
print("=" * 60)

🗓️ RUN THURSDAY: Clean Output Version
📅 2026-01-16 08:55:52

📊 Loading gut scores from MASTER file...
✅ Found 14 total rated albums in master file

📁 Loading training data...
  Current: 10,284 tracks
  Existing gut-scored tracks: 165

🎵 Processing NEW gut-scored albums...

📊 WEEKLY REPORT
ℹ️ No new gut scores to add this week

📈 OVERALL TOTALS:
------------------------------------------------------------
  Total rated albums: 14
  Gut-scored tracks: 165
  Total training tracks: 10,284

✅ THURSDAY PROCESSING COMPLETE


In [3]:
# ====================================================================
# 🎯 THURSDAY: ENHANCED WITH RELEASE DATES
# Shows which NMF week each album came from
# ====================================================================

print("🗓️ RUN THURSDAY: Enhanced with Release Dates")
print("=" * 60)
print("📅 " + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print("=" * 60)

# --- STEP 1: LOAD GUT SCORES FROM MASTER FILE ---
print("\n📊 Loading gut scores from MASTER file...")

master_file = 'feedback/master_gut_scores.csv'

if not os.path.exists(master_file):
    print(f"❌ Master file not found: {master_file}")
    exit()

master_df = pd.read_csv(master_file)
master_df = master_df[master_df['gut_score'].notna()]

if len(master_df) == 0:
    print("❌ No gut scores found!")
    exit()

total_albums = len(master_df)
print(f"✅ Found {total_albums} total rated albums in master file")

# --- STEP 2: LOAD TRAINING DATA ---
print("\n📁 Loading training data...")
training_file = 'data/2026_training_complete_with_features.csv'

if not os.path.exists(training_file):
    print(f"❌ Training file not found: {training_file}")
    exit()

df_training = pd.read_csv(training_file)
print(f"  Current: {len(df_training):,} tracks")

# Count existing gut-scored tracks
if 'source_type' in df_training.columns:
    existing_gut = df_training[df_training['source_type'] == 'gut_score_rated']
    existing_count = len(existing_gut)
    print(f"  Existing gut-scored tracks: {existing_count:,}")
else:
    existing_count = 0

# --- STEP 3: MAP ARCHIVE FILES TO DATES ---
print("\n🗂️  Scanning archive files...")
archive_files = sorted(glob.glob('data/archived_nmf_with_features/*.csv'), reverse=True)

if not archive_files:
    print("❌ No archive files found!")
    exit()

# Create mapping: filename -> human readable date
archive_date_map = {}
for archive_file in archive_files:
    filename = os.path.basename(archive_file)
    # Extract date: "2026-01-11_nmf_complete.csv" -> "2026-01-11"
    date_str = filename.split('_')[0]
    
    try:
        # Convert to readable format: "Jan 11, 2026"
        date_obj = datetime.strptime(date_str, '%Y-%m-%d')
        readable_date = date_obj.strftime('%b %d, %Y')
        archive_date_map[archive_file] = readable_date
    except:
        archive_date_map[archive_file] = date_str

print(f"  Found {len(archive_files)} archive weeks")

# --- STEP 4: PROCESS NEW ALBUMS WITH RELEASE DATES ---
print("\n🎵 Processing NEW gut-scored albums...")

tracks_added = 0
new_albums = 0
new_albums_info = []  # Store (artist, album, score, release_date, track_count)

for _, fb_row in master_df.iterrows():
    artist = fb_row['Artist']
    album = fb_row['Album']
    score = fb_row['gut_score']
    
    # Skip if already in training
    already_in_training = df_training[
        (df_training['Album Name'] == album) &
        (df_training['Artist Name(s)'].str.contains(artist, na=False)) &
        (df_training['source_type'] == 'gut_score_rated')
    ]
    
    if len(already_in_training) > 0:
        continue
    
    # Search archives
    found_tracks = None
    found_archive = None
    release_date = "Unknown"
    
    for archive_file in archive_files:
        try:
            archive_df = pd.read_csv(archive_file)
            album_tracks = archive_df[
                (archive_df['Album Name'] == album) &
                (archive_df['Artist Name(s)'].str.contains(artist, na=False))
            ].copy()
            
            if len(album_tracks) > 0:
                found_tracks = album_tracks
                found_archive = archive_file
                release_date = archive_date_map.get(archive_file, "Unknown")
                break
        except Exception as e:
            continue
    
    if found_tracks is None:
        print(f"  ❌ Could not find tracks for: {artist} - {album}")
        continue
    
    new_albums += 1
    new_albums_info.append({
        'artist': artist,
        'album': album, 
        'score': score,
        'release_date': release_date,
        'tracks': len(found_tracks)
    })
    
    # Add gut score
    found_tracks['liked'] = score
    found_tracks['source_type'] = 'gut_score_rated'
    found_tracks['gut_score_date'] = datetime.now().strftime('%Y-%m-%d')
    found_tracks['nmf_release_date'] = release_date
    
    # Add to training
    df_training = pd.concat([df_training, found_tracks], ignore_index=True)
    tracks_added += len(found_tracks)

# --- STEP 5: SILENT DUPLICATE HANDLING ---
if tracks_added > 0:
    def clean_artist_name(artist_str):
        if pd.isna(artist_str): return ""
        artist = str(artist_str).strip()
        separators = [' feat. ', ' featuring ', ' ft. ', ' with ', ' & ', ' and ', ';', ',']
        for sep in separators:
            if sep in artist.lower():
                artist = artist.split(sep)[0].strip()
        return artist
    
    df_training['artist_clean'] = df_training['Artist Name(s)'].apply(clean_artist_name)
    df_training = df_training.sort_values('liked', ascending=False)
    df_training = df_training.drop_duplicates(
        subset=['Album Name', 'artist_clean', 'Track Name'],
        keep='first'
    )
    df_training = df_training.drop(columns=['artist_clean'], errors='ignore')

# --- STEP 6: ENHANCED REPORT WITH RELEASE DATES ---
print("\n" + "=" * 60)
print("📊 WEEKLY REPORT")
print("=" * 60)

if tracks_added > 0:
    # Create backup
    backup_dir = 'data/backups'
    os.makedirs(backup_dir, exist_ok=True)
    backup_file = f"{backup_dir}/training_backup_{datetime.now().strftime('%Y%m%d')}.csv"
    pd.read_csv(training_file).to_csv(backup_file, index=False)
    
    # Save updated training
    df_training.to_csv(training_file, index=False)
    
    print(f"✅ ADDED THIS WEEK ({len(new_albums_info)} albums):")
    print("-" * 60)
    
    # Group by release date
    albums_by_date = {}
    for info in new_albums_info:
        date = info['release_date']
        if date not in albums_by_date:
            albums_by_date[date] = []
        albums_by_date[date].append(info)
    
    # Show by release date (most recent first)
    for date in sorted(albums_by_date.keys(), reverse=True):
        print(f"\n📅 {date}:")
        for info in albums_by_date[date]:
            print(f"  • {info['artist'][:22]:<22} - {info['album'][:22]:<22}")
            print(f"      Score: {info['score']} | Tracks: {info['tracks']}")
    
    print("-" * 60)
    print(f"  Total tracks added: {tracks_added:,}")
    
else:
    print("ℹ️ No new gut scores to add this week")

# Final stats with breakdown
print(f"\n📈 OVERALL TOTALS:")
print("-" * 60)

# Get all gut-scored tracks
if 'nmf_release_date' in df_training.columns:
    gut_scored = df_training[df_training['source_type'] == 'gut_score_rated']
    
    if len(gut_scored) > 0:
        # Count by release date
        date_counts = gut_scored['nmf_release_date'].value_counts()
        
        print(f"  Gut-scored tracks by release week:")
        for date, count in date_counts.head(5).items():  # Show top 5
            print(f"    {date}: {count:,} tracks")
        
        if len(date_counts) > 5:
            print(f"    ... and {len(date_counts) - 5} more weeks")
    
    print(f"  Total gut-scored tracks: {len(gut_scored):,}")
else:
    gut_scored = df_training[df_training['source_type'] == 'gut_score_rated']
    print(f"  Total gut-scored tracks: {len(gut_scored):,}")

print(f"  Total rated albums: {total_albums}")
print(f"  Total training tracks: {len(df_training):,}")

print("\n" + "=" * 60)
print("✅ THURSDAY PROCESSING COMPLETE")
print("=" * 60)

🗓️ RUN THURSDAY: Enhanced with Release Dates
📅 2026-01-16 08:55:53

📊 Loading gut scores from MASTER file...
✅ Found 14 total rated albums in master file

📁 Loading training data...
  Current: 10,284 tracks
  Existing gut-scored tracks: 165

🗂️  Scanning archive files...
  Found 1 archive weeks

🎵 Processing NEW gut-scored albums...

📊 WEEKLY REPORT
ℹ️ No new gut scores to add this week

📈 OVERALL TOTALS:
------------------------------------------------------------
  Total gut-scored tracks: 165
  Total rated albums: 14
  Total training tracks: 10,284

✅ THURSDAY PROCESSING COMPLETE
